In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv')


# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [3]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_pos_fs.sav', 'wb'))

In [4]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29111
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21179
[3]	valid_0's multi_logloss: 1.14494
[4]	valid_0's multi_logloss: 1.08447
[5]	valid_0's multi_logloss: 1.03079
[6]	valid_0's multi_logloss: 0.984404
[7]	valid_0's multi_logloss: 0.941434
[8]	valid_0's multi_logloss: 0.904368
[9]	valid_0's multi_logloss: 0.870758
[10]	valid_0's multi_logloss: 0.840239
[11]	valid_0's multi_logloss: 0.811879
[12]	valid_0's multi_logloss: 0.786344
[13]	valid_0's multi_logloss: 0.763216
[14]	valid_0's multi_logloss: 0.740627
[15]	valid_0's multi_logloss: 0.721401
[16]	valid_0's multi_logloss: 0.704555
[17]	valid_0's multi_logloss: 0.687463
[18]	valid_0's multi_logloss: 0.673109
[19]	valid_0's multi_logloss: 0.657522
[20]	valid_0's multi_logloss: 0.641625
[21]	valid_0's multi_logloss: 0.628216
[22]	valid_0's multi_logloss: 0.617631
[23]	valid_0's multi_logloss: 0.60539
[24]	valid_0's multi_logloss: 0.593481
[25]	valid_0's mul

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [6]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
pickle.dump(model, open('../model/xgb_pos_fs.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 2ms/step - loss: 1.8072 - acc: 0.2567 - val_loss: 1.4054 - val_acc: 0.2232
Epoch 2/100
1340/1340 [==============================] - 0s 75us/step - loss: 1.7618 - acc: 0.2784 - val_loss: 1.3839 - val_acc: 0.2946
Epoch 3/100
1340/1340 [==============================] - 0s 75us/step - loss: 1.7601 - acc: 0.2739 - val_loss: 1.3674 - val_acc: 0.3036
Epoch 4/100
1340/1340 [==============================] - 0s 78us/step - loss: 1.6409 - acc: 0.3022 - val_loss: 1.3427 - val_acc: 0.3274
Epoch 5/100
1340/1340 [==============================] - 0s 75us/step - loss: 1.5728 - acc: 0.3097 - val_loss: 1.2999 - val_acc: 0.3393
Epoch 6/100
1340/1340 [==============================] - 0s 85us/step - loss: 1.5519 - acc: 0.3440 - val_loss: 1.2940 - val_acc: 0.3661
Epoch 7/100
1340/1340 [==============================] - 0s 80us/step - loss: 1.5606 - acc: 0.3366 - val_loss: 1.2849 - val_acc: 0.4345
Ep

1340/1340 [==============================] - 0s 85us/step - loss: 0.6320 - acc: 0.7366 - val_loss: 0.8695 - val_acc: 0.6696
Epoch 61/100
1340/1340 [==============================] - 0s 78us/step - loss: 0.6242 - acc: 0.7358 - val_loss: 0.8800 - val_acc: 0.6696
Epoch 62/100
1340/1340 [==============================] - 0s 78us/step - loss: 0.6098 - acc: 0.7373 - val_loss: 0.8930 - val_acc: 0.6458
Epoch 63/100
1340/1340 [==============================] - 0s 76us/step - loss: 0.5618 - acc: 0.7687 - val_loss: 0.8980 - val_acc: 0.6696
Epoch 64/100
1340/1340 [==============================] - 0s 77us/step - loss: 0.5779 - acc: 0.7612 - val_loss: 0.8829 - val_acc: 0.6935
Epoch 65/100
1340/1340 [==============================] - 0s 78us/step - loss: 0.5679 - acc: 0.7642 - val_loss: 0.8697 - val_acc: 0.6667
Epoch 66/100
1340/1340 [==============================] - 0s 81us/step - loss: 0.5484 - acc: 0.7791 - val_loss: 0.8675 - val_acc: 0.6399
Epoch 67/100
1340/1340 [==============================

In [8]:
f = []
f.append(t)

In [9]:
f

[[75.3, 85.71, 80.95, 66.37]]

In [10]:
#read data
df = pd.read_csv('../data/feature_selection_negative.csv')

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [11]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
print(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_ng_fs.sav', 'wb'))

73.4


In [12]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

# pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))
# result append to list
t.append(
round(gbm.score(X_test, y_test)*100, 2)
)
pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))

[1]	valid_0's multi_logloss: 1.31022
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24176
[3]	valid_0's multi_logloss: 1.18146
[4]	valid_0's multi_logloss: 1.13556
[5]	valid_0's multi_logloss: 1.08965
[6]	valid_0's multi_logloss: 1.05396
[7]	valid_0's multi_logloss: 1.01492
[8]	valid_0's multi_logloss: 0.980618
[9]	valid_0's multi_logloss: 0.949026
[10]	valid_0's multi_logloss: 0.917333
[11]	valid_0's multi_logloss: 0.895576
[12]	valid_0's multi_logloss: 0.875062
[13]	valid_0's multi_logloss: 0.851414
[14]	valid_0's multi_logloss: 0.833152
[15]	valid_0's multi_logloss: 0.81752
[16]	valid_0's multi_logloss: 0.803891
[17]	valid_0's multi_logloss: 0.791393
[18]	valid_0's multi_logloss: 0.780679
[19]	valid_0's multi_logloss: 0.768491
[20]	valid_0's multi_logloss: 0.760004
[21]	valid_0's multi_logloss: 0.751434
[22]	valid_0's multi_logloss: 0.740741
[23]	valid_0's multi_logloss: 0.73175
[24]	valid_0's multi_logloss: 0.726565
[25]	valid_0's multi_

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [14]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
pickle.dump(model, open('../model/xgb_ng_fs.sav', 'wb'))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 6ms/step - loss: 2.3022 - acc: 0.1903 - val_loss: 1.4322 - val_acc: 0.4255
Epoch 2/100
373/373 [==============================] - 0s 61us/step - loss: 2.0234 - acc: 0.2895 - val_loss: 1.4182 - val_acc: 0.4681
Epoch 3/100
373/373 [==============================] - 0s 60us/step - loss: 1.9597 - acc: 0.2895 - val_loss: 1.5056 - val_acc: 0.4894
Epoch 4/100
373/373 [==============================] - 0s 66us/step - loss: 1.6628 - acc: 0.3780 - val_loss: 1.5702 - val_acc: 0.5106
Epoch 5/100
373/373 [==============================] - 0s 55us/step - loss: 1.6731 - acc: 0.3727 - val_loss: 1.5797 - val_acc: 0.5106
Epoch 6/100
373/373 [==============================] - 0s 62us/step - loss: 1.6356 - acc: 0.3753 - val_loss: 1.5369 - val_acc: 0.5000
Epoch 7/100
373/373 [==============================] - 0s 52us/step - loss: 1.7144 - acc: 0.3673 - val_loss: 1.4960 - val_acc: 0.5000
Epoch 8/100
373/37

Epoch 62/100
373/373 [==============================] - 0s 62us/step - loss: 0.6610 - acc: 0.7239 - val_loss: 1.0584 - val_acc: 0.6277
Epoch 63/100
373/373 [==============================] - 0s 57us/step - loss: 0.6616 - acc: 0.7212 - val_loss: 1.0619 - val_acc: 0.6170
Epoch 64/100
373/373 [==============================] - 0s 77us/step - loss: 0.6500 - acc: 0.7373 - val_loss: 1.0678 - val_acc: 0.6170
Epoch 65/100
373/373 [==============================] - 0s 72us/step - loss: 0.6705 - acc: 0.6971 - val_loss: 1.0733 - val_acc: 0.6064
Epoch 66/100
373/373 [==============================] - 0s 62us/step - loss: 0.6454 - acc: 0.7480 - val_loss: 1.0832 - val_acc: 0.6064
Epoch 67/100
373/373 [==============================] - 0s 69us/step - loss: 0.6401 - acc: 0.7480 - val_loss: 1.0934 - val_acc: 0.6064
Epoch 68/100
373/373 [==============================] - 0s 69us/step - loss: 0.6263 - acc: 0.7507 - val_loss: 1.1027 - val_acc: 0.6064
Epoch 69/100
373/373 [==============================] -

In [16]:
f.append(t)

In [17]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Feature_Selection_Positive', 'Feature_Selection_Negative']
)

,Random Forest,LightGBM,XGBoost,Keras
Feature_Selection_Positive,75.3,85.71,80.95,66.37
Feature_Selection_Negative,73.4,75.53,72.34,63.83


In [18]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Feature_Selection_Positive', 'Feature_Selection_Negative']
).to_csv('../result/Feature_Selection_result.csv')